In [1]:
import pandas as pd
from src.data import MovieLens20MDataLoader
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
ml_loader = MovieLens20MDataLoader()
ml_data = ml_loader.load()

file is found. loading...


In [3]:
print(ml_data.keys())
#display(ml_data.get('genome-scores').head())
#display(ml_data.get('genome-tags').head())
display(ml_data.get('movies').head())
display(ml_data.get('ratings').head())

dict_keys(['genome-scores', 'genome-tags', 'links', 'movies', 'ratings', 'tags'])


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [4]:
# ユーザーごとにチャンク読み込み、結合

_movies = ml_data.get('movies').copy()
_ratings = ml_data.get('ratings').copy()

save_path = f"{MovieLens20MDataLoader().DATA_DIRECTORY}/{MovieLens20MDataLoader().directory_name}/ratings"
os.makedirs(save_path, exist_ok=True)

_size = 20
_loop = len(_ratings.userId.unique().tolist()) // _size + 1

user_limit = 2 # _size * user_limit を上限とする
movie_limit = 40


for index in tqdm(range(_loop)):
    if user_limit * index > user_limit:
        break
    start = index * _size
    end = (index + 1) * _size
    file_name = f"{save_path}/rating_matrix_{start}-{end}.csv"
    _ratings.loc[
        (_ratings["userId"].isin(range(start, end)))
        & (_ratings["movieId"] < movie_limit)
    ].pivot_table(values=["rating"], index=["userId"], columns=["movieId"]).to_csv(file_name)


matrix_data = pd.DataFrame()
for index in tqdm(range(_loop)):
    if user_limit * index > user_limit:
        break
    start = index * _size
    end = (index + 1) * _size
    file_name = f"{save_path}/rating_matrix_{start}-{end}.csv"
    _data = pd.read_csv(file_name, header=1, index_col=0)
    _data = _data.drop("userId") # 謎の行を削除
    matrix_data = pd.concat([matrix_data, _data], axis=0)
    
matrix_data.to_csv(f"{save_path}/rating_matrix_all.csv")

/tmp/ipykernel_246/1875492515.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm(range(_loop)):


  0%|          | 0/6925 [00:00<?, ?it/s]

/tmp/ipykernel_246/1875492515.py:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index in tqdm(range(_loop)):


  0%|          | 0/6925 [00:00<?, ?it/s]

In [13]:
matrix_data2 = pd.read_csv(f"{save_path}/rating_matrix_all.csv", index_col=0)
matrix_data2 = matrix_data2.fillna(0)
matrix_data2

,1,2,3,5,6,7,10,11,14,15,...,34,36,39,8,9,18,20,22,23,31
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,3.0,0.0,0.0,3.0,0.0,4.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4.0,0.0,5.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# https://qiita.com/kidaufo/items/0f3da4ca4e19dc0e987e

import numpy as np
from numpy.linalg import svd, matrix_rank

# rank=2の　３ｘ4行列を作成
A = matrix_data2 #np.array([[2, 4, 1, 3], [1, 5, 3, 2], [5, 7, 0, 7]])

print('matrix A\n', A)
print('rank: ', matrix_rank(A))

# singular value decomposition
u, s, vh = svd(A)
print('\nSVD result')
print('shape of u, s, vh:', u.shape, s.shape, vh.shape)
print('singular values:', s.round(2))
print("u\n", u, "\ns\n", s, "\nvh\n", vh)

# full_matrices=Falseの場合
u, s, vh = svd(A, full_matrices=False)
print('\nSVD result　(full_matrices: False)')

# 復元
# A_re = (u @ np.diag(s, -1)[1:] @ vh).round(2)
A_re = (u @ np.diag(s) @ vh).round(2)
print('\nreconstructed A:\n', A_re)


matrix A
            1    2    3    5    6    7   10   11   14   15  ...   34   36   39  \
movieId                                                    ...                  
1        0.0  3.5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2        0.0  0.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3        4.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4        0.0  0.0  0.0  0.0  3.0  0.0  4.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
5        0.0  3.0  0.0  0.0  0.0  0.0  0.0  5.0  0.0  0.0  ...  0.0  0.0  0.0   
6        5.0  0.0  3.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
7        0.0  0.0  3.0  0.0  0.0  3.0  0.0  4.0  0.0  2.0  ...  0.0  0.0  0.0   
8        4.0  0.0  5.0  0.0  3.0  0.0  4.0  0.0  0.0  0.0  ...  0.0  0.0  3.0   
10       4.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
11       4.5  0.0  0.0  0.0  0.0  0.0  2.5  0.0  0.0  0.0  ...  0.0  0.0  4.5   
12       4.0  0.0 

In [17]:
# MF

matrix_data2 = pd.read_csv(f"{save_path}/rating_matrix_all.csv", index_col=0)
matrix_data2

,1,2,3,5,6,7,10,11,14,15,...,34,36,39,8,9,18,20,22,23,31
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5.0,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,3.0,NaN,NaN,3.0,NaN,4.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4.0,NaN,5.0,NaN,3.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
import scipy
from scipy.sparse.linalg import svds
import numpy as np

user_id_map = dict(zip(matrix_data2.index, range(len(matrix_data2.index))))
movie_id_map = dict(zip(matrix_data2.columns, range(len(matrix_data2.columns))))

matrix = matrix_data2.copy()
matrix = matrix.fillna(matrix_data2.mean()).to_numpy()

# 因子数kで分解
P, S, Qt = svds(matrix, k=5)

# 予測値評価行列
pred_matrix = np.dot(np.dot(P, np.diag(S)), Qt)

print(f"P: {P.shape}, S: {S.shape}, Qt: {Qt.shape}, pred_matrix: {pred_matrix.shape}")

pred_matrix

P: (32, 5), S: (5,), Qt: (5, 28), pred_matrix: (32, 28)


array([[4.14005735, 3.11970533, 3.59065167, 2.17796869, 3.83121659,
        3.83208165, 3.29432308, 4.02231577, 3.0171072 , 3.02058234,
        4.16384711, 3.77278452, 2.76353855, 3.19959227, 3.26943378,
        4.02324402, 3.26921292, 3.52605543, 3.44283876, 3.52153702,
        3.57815356, 3.01755197, 4.02340262, 5.02925328, 4.02340262,
        3.35168999, 4.02340262, 3.35537389],
       [4.12731987, 3.11162924, 3.62007409, 2.17416072, 3.77782703,
        3.79497628, 3.34442238, 4.01956358, 3.00910622, 3.02144949,
        4.1649022 , 3.83077984, 2.72287177, 3.24762525, 3.26795929,
        4.02873119, 3.26282278, 3.57297265, 3.39748742, 3.50485972,
        3.55567163, 3.01176545, 4.01568727, 5.01960909, 4.01568727,
        3.34606889, 4.01568727, 3.34563831],
       [4.06088369, 3.09655608, 3.56133537, 2.16601244, 3.80476424,
        3.75405628, 3.20705098, 3.99430803, 2.95506712, 2.98865698,
        4.1725317 , 3.78209796, 2.76884542, 3.18942768, 3.26288717,
        4.05483454, 3.2494

In [34]:
def user_index2id(index):
    for k, v in user_id_map.items():
        if index == v:
            return k
def user_id2index(id):
    return user_id_map[id]
def movie_index2id(index):
    for k, v in movie_id_map.items():
        if index == v:
            return k
def movie_id2index(id):
    return movie_id_map[id]

In [52]:
recommend_user_id = 1

user_index = user_id2index(recommend_user_id)


result = pd.DataFrame(
    pred_matrix[user_index],
    columns=["score"],
    index=movie_id_map.keys()
)
result.index.name = "movieId"
result.sort_values(by="score", ascending=False)

,score
movieId,
18,5.029253
16,4.163847
1,4.140057
23,4.023403
20,4.023403
9,4.023403
25,4.023244
11,4.022316
7,3.832082
